In [1]:
#importing libaries 

import numpy as np 
import matplotlib as plt
import seaborn as sns 
import pandas as pd
import plotly.express as px
from plotly.offline import iplot
from plotly.subplots import make_subplots

In [2]:
data = pd.read_csv("IMDb Movies India.csv",encoding='ISO-8859-1')

In [3]:
data.head()

,Name,Year,Duration,Genre,Rating,Votes,Director,Actor 1,Actor 2,Actor 3
0,,NaN,NaN,Drama,NaN,NaN,J.S. Randhawa,Manmauji,Birbal,Rajendra Bhatia
1,#Gadhvi (He thought he was Gandhi),(2019),109 min,Drama,7.0,8,Gaurav Bakshi,Rasika Dugal,Vivek Ghamande,Arvind Jangid
2,#Homecoming,(2021),90 min,"Drama, Musical",NaN,NaN,Soumyajit Majumdar,Sayani Gupta,Plabita Borthakur,Roy Angana
3,#Yaaram,(2019),110 min,"Comedy, Romance",4.4,35,Ovais Khan,Prateik,Ishita Raj,Siddhant Kapoor
4,...And Once Again,(2010),105 min,Drama,NaN,NaN,Amol Palekar,Rajat Kapoor,Rituparna Sengupta,Antara Mali


In [5]:
data.describe()

,Rating
count,7919.000000
mean,5.841621
std,1.381777
min,1.100000
25%,4.900000
50%,6.000000
75%,6.800000
max,10.000000


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15509 entries, 0 to 15508
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Name      15509 non-null  object 
 1   Year      14981 non-null  object 
 2   Duration  7240 non-null   object 
 3   Genre     13632 non-null  object 
 4   Rating    7919 non-null   float64
 5   Votes     7920 non-null   object 
 6   Director  14984 non-null  object 
 7   Actor 1   13892 non-null  object 
 8   Actor 2   13125 non-null  object 
 9   Actor 3   12365 non-null  object 
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


In [7]:
data.isnull().any()

Name        False
Year         True
Duration     True
Genre        True
Rating       True
Votes        True
Director     True
Actor 1      True
Actor 2      True
Actor 3      True
dtype: bool

In [8]:
data = data.dropna(subset=["Year","Duration","Genre","Rating","Votes","Director","Actor 1", "Actor 2", "Actor 3"])

In [9]:
data = data.dropna().reset_index(drop=True)

In [10]:
data.isnull().any()

Name        False
Year        False
Duration    False
Genre       False
Rating      False
Votes       False
Director    False
Actor 1     False
Actor 2     False
Actor 3     False
dtype: bool

In [11]:
data.duplicated(subset=["Name","Year"]).sum()

2

In [12]:
data=data.drop_duplicates(keep="first",subset=["Name","Year"]).reset_index(drop=True)

In [13]:
data["Genre"] = data.loc[:,"Genre"].str.replace("Musical","Music")

In [14]:
data["Main_genre"] = data.loc[:,"Genre"].str.extract("(^\w{1,11})")

<>:1: SyntaxWarning:

invalid escape sequence '\w'

<>:1: SyntaxWarning:

invalid escape sequence '\w'

C:\Users\hp\AppData\Local\Temp\ipykernel_7712\583801436.py:1: SyntaxWarning:

invalid escape sequence '\w'



In [15]:
data["Year"] = data.loc[:,"Year"].str.extract(r"(\d{4})")
data["Year"] = pd.to_numeric(data.loc[:,"Year"])

In [16]:
data["Main_genre"].unique()

array(['Drama', 'Comedy', 'Horror', 'Action', 'Crime', 'Thriller',
       'Adventure', 'Sport', 'Biography', 'Documentary', 'Mystery',
       'Music', 'Romance', 'Fantasy', 'Sci', 'Family', 'History',
       'Animation', 'War'], dtype=object)

In [17]:
genres = data.Main_genre.value_counts().reset_index()
iplot(px.pie(data_frame=genres,names=genres.Main_genre,values=genres["count"],title="Number of movies by Genre",height=1050)\
      .update_traces(textinfo="value+percent"))

In [18]:
rating_by_genre = data.groupby("Main_genre")["Rating"].mean().sort_values(ascending = False)
iplot(px.bar(data_frame=rating_by_genre))

In [19]:
data["Duration"] = data.loc[:,"Duration"].str.replace(" min","")
data["Duration"] = pd.to_numeric(data.loc[:,"Duration"])

In [20]:
data["Votes"] = data.loc[:,"Votes"].str.replace(",","")
data["Votes"] = pd.to_numeric(data.loc[:,"Votes"])

In [21]:
from sklearn.model_selection import train_test_split,cross_val_score
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.preprocessing import RobustScaler,StandardScaler,MaxAbsScaler,MinMaxScaler
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
import sklearn.metrics as metrics
import category_encoders as ce

In [22]:
data =data.drop(columns=["Name","Main_genre"])

In [23]:
X = data.drop(columns="Rating")
y = data["Rating"]

In [24]:
encoder = ce.JamesSteinEncoder(return_df=True)

In [25]:
encoder.fit(X,y)
X = encoder.transform(X.loc[:,:])

In [26]:
scaler = RobustScaler()
scaler.fit(X)
X.loc[:,:] = scaler.transform(X.loc[:,:])

C:\Users\hp\AppData\Local\Temp\ipykernel_7712\1096380571.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.56666667  0.56666667 -0.16666667 ... -0.43333333 -0.1
 -0.13333333]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\hp\AppData\Local\Temp\ipykernel_7712\1096380571.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.83870968 -0.80645161  0.38709677 ... -0.32258065 -0.19354839
 -0.16129032]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

C:\Users\hp\AppData\Local\Temp\ipykernel_7712\1096380571.py:3: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.13773796 -0.1075028   0.7793953  ... -0.09742441  0.58678611
 -0.12430011]' has dtype incompatible with int64, please exp

In [27]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=7,shuffle=True)

In [28]:
xgb_model = XGBRegressor(objective ='reg:squarederror',gamma=0.09,learning_rate=0.08,subsample=0.7)

In [29]:
xgb_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.09, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.08, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [30]:
xgb_model.score(X_train,y_train)

0.927784858654971

In [31]:
xgb_model.score(X_test,y_test)

0.8475258356668441